# High-level PyTorch Example

In [1]:
# Parameters
EPOCHS = 10
N_CLASSES=10
BATCHSIZE = 64
LR = 0.01
MOMENTUM = 0.9
GPU = True

LOGGER_URL='msdlvm.southcentralus.cloudapp.azure.com'
LOGGER_USRENAME='admin'
LOGGER_PASSWORD='password'
LOGGER_DB='gpudata'
LOGGER_SERIES='gpu'

In [2]:
import os
from os import path
import sys
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
import torch.nn.init as init
from torch.autograd import Variable
from utils import cifar_for_library, yield_mb, create_logger, Timer
from nb_logging import NotebookLogger, output_to, error_to
from gpumon.influxdb import log_context
import codecs

from influxdb import InfluxDBClient

In [4]:
client = InfluxDBClient(LOGGER_URL, 8086, LOGGER_USRENAME, LOGGER_PASSWORD, LOGGER_DB)

In [5]:
node_id = os.getenv('AZ_BATCH_NODE_ID', default='node')
task_id = os.getenv('AZ_BATCH_TASK_ID', default='pytorch')
job_id = os.getenv('AZ_BATCH_JOB_ID', default='pytorch')

In [6]:
logger = create_logger(client, node_id=node_id, task_id=task_id, job_id=job_id)

In [7]:
sys.__stdout__ = codecs.getwriter("utf-8")(sys.__stdout__.detach())

In [8]:
nb_teminal_logger = NotebookLogger(sys.stdout.session, sys.stdout.pub_thread, sys.stdout.name, sys.__stdout__)

In [9]:
rst_out = output_to(nb_teminal_logger)
rst_err = error_to(nb_teminal_logger)

INFO:gpumon.influxdb_gpu_logger:Logging GPU to Database msdlvm.southcentralus.cloudapp.azure.com
INFO:gpumon.influxdb_gpu_logger:['influxdb_gpu_logger.py', 'msdlvm.southcentralus.cloudapp.azure.com', '8086', 'admin', 'password', 'gpudata', 'gpu', '--node_id=node', '--task_id=pytorch', '--job_id=pytorch']


In [ ]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("PyTorch: ", torch.__version__)
print("Numpy: ", np.__version__)

In [10]:
# Big impact on training-time (from 350 to 165s)
torch.backends.cudnn.benchmark=True # enables cudnn's auto-tuner

In [11]:
data_path = path.join(os.getenv('AZ_BATCHAI_INPUT_DATASET'), 'cifar-10-batches-py')

In [12]:
class SymbolModule(nn.Module):
    def __init__(self):
        super(SymbolModule, self).__init__()
        self.conv1 = nn.Conv2d(3, 50, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(50, 50, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(50, 100, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(100, 100, kernel_size=3, padding=1)
        # feature map size is 8*8 by pooling
        self.fc1 = nn.Linear(100*8*8, 512)
        self.fc2 = nn.Linear(512, N_CLASSES)

    def forward(self, x):
        """ PyTorch requires a flag for training in dropout """
        x = self.conv2(F.relu(self.conv1(x)))
        x = F.relu(F.max_pool2d(x, kernel_size=2, stride=2))
        x = F.dropout(x, 0.25, training=self.training)

        x = self.conv4(F.relu(self.conv3(x)))
        x = F.relu(F.max_pool2d(x, kernel_size=2, stride=2))
        x = F.dropout(x, 0.25, training=self.training)

        x = x.view(-1, 100*8*8)   # reshape Variable
        x = F.dropout(F.relu(self.fc1(x)), 0.5, training=self.training)
        # nn.CrossEntropyLoss() contains softmax, don't apply twice
        #return F.log_softmax(x)
        return self.fc2(x)

In [13]:
def init_model(m):
    # Implementation of momentum:
    # v = \rho * v + g \\
    # p = p - lr * v
    opt = optim.SGD(m.parameters(), lr=LR, momentum=MOMENTUM)
    criterion = nn.CrossEntropyLoss()
    return opt, criterion

In [14]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(data_path, channel_first=True)
# Torch-specific
y_train = y_train.astype(np.int64)
y_test = y_test.astype(np.int64)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
Done.
(50000, 3, 32, 32) (10000, 3, 32, 32) (50000,) (10000,)
float32 float32 int64 int64
CPU times: user 980 ms, sys: 880 ms, total: 1.86 s
Wall time: 1.86 s


In [15]:
%%time
sym = SymbolModule()
sym.cuda() # CUDA!

CPU times: user 1.76 s, sys: 2.79 s, total: 4.55 s
Wall time: 9.23 s


In [16]:
%%time
optimizer, criterion = init_model(sym)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 210 µs


In [17]:
with Timer() as t:
    with log_context(LOGGER_URL, 'admin', LOGGER_USRENAME, LOGGER_PASSWORD, LOGGER_DB, LOGGER_SERIES, 
                     node_id=node_id, task_id=task_id, job_id=job_id):
        # Sets training = True
        sym.train()  
        for j in range(EPOCHS):
            for data, target in yield_mb(x_train, y_train, BATCHSIZE, shuffle=True):
                # Get samples
                data = Variable(torch.FloatTensor(data).cuda())
                target = Variable(torch.LongTensor(target).cuda())
                # Init
                optimizer.zero_grad()
                # Forwards
                output = sym(data)
                # Loss
                loss = criterion(output, target)
                # Back-prop
                loss.backward()
                optimizer.step()
            # Log
            print(j)
print('Training took %.03f sec.' % t.interval)
logger('training duration', value=t.interval)

0
1
2
3
4
5
6
7
8
9
Training took 107.903 sec.


In [18]:
%%time
# Test model
# Sets training = False
sym.eval()
n_samples = (y_test.shape[0]//BATCHSIZE)*BATCHSIZE
y_guess = np.zeros(n_samples, dtype=np.int)
y_truth = y_test[:n_samples]
c = 0
for data, target in yield_mb(x_test, y_test, BATCHSIZE):
    # Get samples
    data = Variable(torch.FloatTensor(data).cuda())
    # Forwards
    output = sym(data)
    pred = output.data.max(1)[1].cpu().numpy().squeeze()
    # Collect results
    y_guess[c*BATCHSIZE:(c+1)*BATCHSIZE] = pred
    c += 1

CPU times: user 400 ms, sys: 232 ms, total: 632 ms
Wall time: 631 ms


In [19]:
acc=sum(y_guess == y_truth)/len(y_guess)
print("Accuracy: ", acc)
logger('accuracy', value=acc)

Accuracy:  0.777243589744
